### Importing necessary library

In [5]:
import pandas as pd
import numpy as np

import datetime

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

### Reading the cleaned data file

In [11]:
rent_df = pd.read_csv('../Data/canada_rent_clean.csv')
rent_df.head(5)

,city,province,latitude,longitude,lease_term,type,price,beds,baths,sq_feet,furnishing,availability_date,smoking,cats,dogs,pet_category
0,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2495,2,2.5,1403,Unfurnished,2025-02-21,Non-Smoking,True,True,Dogs Allowed
1,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2695,3,2.5,1496,Unfurnished,2025-02-21,Non-Smoking,True,True,Dogs Allowed
2,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2295,2,2.5,1180,Unfurnished,2025-02-21,Non-Smoking,True,True,Dogs Allowed
3,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2095,2,2.5,1403,Unfurnished,2025-11-18,Non-Smoking,True,True,Dogs Allowed
4,Airdrie,Alberta,51.305962,-114.012515,Long Term,Townhouse,2495,2,2.5,1403,Unfurnished,2025-02-21,Non-Smoking,True,True,Dogs Allowed


### Encoding categorical columns

---
As we have seen previously in the Data_Analysis.ipynb, there are six nominal category columns.
  * province
  * lease_term
  * type
  * furnishing
  * smoking
  * pet_category

In [12]:
# Create instance of OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse_output = False)

# Encode the categorical columns
encoded_cols = encoder.fit_transform(rent_df[['province', 'lease_term', 'type',
                                              'furnishing', 'smoking', 'pet_category']])

# Put the encoded column into a dataframe
tmp_df = pd.DataFrame(encoded_cols, columns = encoder.get_feature_names_out(['province',
                                                                             'lease_term', 
                                                                             'type',
                                                                             'furnishing',
                                                                             'smoking', 
                                                                             'pet_category']))

# Removing categorical columns
rent_df = rent_df.drop(columns=['province', 'lease_term', 'type',
                                'furnishing', 'smoking', 'pet_category'])

# Join encoded dataframe to the rent_df dataframe
rent_df = rent_df.join(tmp_df)

rent_df.head()

,city,latitude,longitude,price,beds,baths,sq_feet,availability_date,cats,dogs,...,type_Main Floor,type_Mobile,type_Room For Rent,type_Townhouse,type_Vacation Home,furnishing_Negotiable,furnishing_Unfurnished,smoking_Smoking Allowed,pet_category_Dogs Allowed,pet_category_no pets
0,Airdrie,51.305962,-114.012515,2495,2,2.5,1403,2025-02-21,True,True,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,Airdrie,51.305962,-114.012515,2695,3,2.5,1496,2025-02-21,True,True,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,Airdrie,51.305962,-114.012515,2295,2,2.5,1180,2025-02-21,True,True,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,Airdrie,51.305962,-114.012515,2095,2,2.5,1403,2025-11-18,True,True,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,Airdrie,51.305962,-114.012515,2495,2,2.5,1403,2025-02-21,True,True,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


### Removing unecessary columns
---
As we have seen previously in the Cleaning_Data.ipynb, we well not be working with city name, since it would create tons of encoded columns. For simplicity, we will be working with latitude and longitude.

So, I will be removing the city column at this point.

---

I also will be removing the availability_date column, since I don't believe that this can affect the price.

In [13]:
rent_df = rent_df.drop(columns = ['availability_date', 'city'])

### Splitting the dataset into training vs testing dataset

In [3]:
train_df = 

test_df = 

24419

In [14]:
len(rent_df.columns)

37